### Topic Modeling

One Company

In [21]:
df_MSFT = df_Z[df_Z['Company'] == 'MSFP']

In [7]:
from inscriptis import get_text
import glob
from lxml import html
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim import corpora
import pandas as pd

In [8]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [9]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

def clean(doc):
    temp = doc.replace('\n', ' ')
    temp = temp.replace('\xa0', ' ')
    temp = temp.replace('\x94', ' ')
    temp = temp.replace('\x92', ' ')
    temp = temp.replace('\x97', ' ')
    temp = temp.replace('\x93', ' ')
    temp = temp.replace('\x99', ' ')
    stop_free = " ".join([i for i in temp.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    no_num = ''.join([i for i in normalized if not i.isdigit()])
    singleletter_free = ' '.join( [w for w in no_num.split() if len(w)>1] )
    no_sth = ' '.join( [w for w in singleletter_free.split() if w != 'th'] )
    no_ic = ' '.join( [w for w in no_sth.split() if w != 'ic'] )

    return no_ic

In [32]:
MSFT_content = df_MSFT['Content'].tolist()
doc_clean = [clean(doc).split() for doc in MSFT_content]  

In [37]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean]

In [46]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50, minimum_probability=0)

In [47]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      51.746387        1       1 -0.097129 -0.112882
1      36.994251        1       2 -0.150647  0.097719
2      11.259361        1       3  0.247775  0.015163, topic_info=     Category         Freq               Term        Total  loglift  logprob
term                                                                        
14    Default   741.000000           question   741.000000  30.0000  30.0000
10    Default   606.000000               next   606.000000  29.0000  29.0000
152   Default   604.000000             thanks   604.000000  28.0000  28.0000
22    Default   373.000000                you   373.000000  27.0000  27.0000
19    Default   364.000000              thank   364.000000  26.0000  26.0000
112   Default   285.000000               come   285.000000  25.0000  25.0000
113   Default   312.000000              great   312.000000  24.0000  24.0000
114   Default   242.000000               line   242.000000  23.0000  23.0000
13    Default   730.000000            quarter   730.000000  22.0000  22.0000
612   Default   137.000000               okay   137.000000  21.0000  21.0000
109   Default   582.000000               year   582.000000  20.0000  20.0000
128   Default   310.000000              cloud   310.000000  19.0000  19.0000
458   Default    87.000000           operator    87.000000  18.0000  18.0000
146   Default   232.000000               name   232.000000  17.0000  17.0000
137   Default   139.000000                guy   139.000000  16.0000  16.0000
117   Default    83.000000             please    83.000000  15.0000  15.0000
740   Default    77.000000               open    77.000000  14.0000  14.0000
623   Default   134.000000                 hi   134.000000  13.0000  13.0000
205   Default   367.000000            revenue   367.000000  12.0000  12.0000
145   Default   224.000000               much   224.000000  11.0000  11.0000
392   Default   212.000000              azure   212.000000  10.0000  10.0000
136   Default   452.000000              going   452.000000   9.0000   9.0000
101   Default  1134.000000              think  1134.000000   8.0000   8.0000
71    Default   253.000000                lot   253.000000   7.0000   7.0000
126   Default   731.000000           business   731.000000   6.0000   6.0000
470   Default   228.000000             impact   228.000000   5.0000   5.0000
2524  Default   128.000000              chris   128.000000   4.0000   4.0000
534   Default    50.000000  pleaseoperatorour    50.000000   3.0000   3.0000
484   Default   104.000000          wondering   104.000000   2.0000   2.0000
2531  Default   114.000000              vista   114.000000   1.0000   1.0000
...       ...          ...                ...          ...      ...      ...
2237   Topic3    13.477031      pleasecompany    14.215879   2.1306  -6.5661
3470   Topic3    12.517421         youcompany    13.245486   2.1274  -6.6400
2582   Topic3    11.754451              friar    12.460959   2.1256  -6.7029
19     Topic3   320.524353              thank   364.188690   2.0563  -3.3971
612    Topic3   121.228500               okay   137.672623   2.0568  -4.3694
117    Topic3    73.987328             please    83.123489   2.0675  -4.8632
14     Topic3   543.917969           question   741.940796   1.8735  -2.8683
10     Topic3   439.510162               next   606.532654   1.8619  -3.0814
112    Topic3   176.597488               come   285.974121   1.7019  -3.9932
152    Topic3   327.419556             thanks   604.023193   1.5716  -3.3759
114    Topic3   142.540985               line   242.111694   1.6542  -4.2075
1497   Topic3    27.213051           terrific    33.629620   1.9723  -5.8634
113    Topic3   141.886917              great   312.791809   1.3935  -4.2121
828    Topic3    21.757673                hey    25.967770   2.0071  -6.0871
532    Topic3    30.498814               john    43.598232   1.8266 

### Biotech Industry

Try with all texts within earnings call transcript

In [13]:
df_bio = pd.read_csv('/Users/sharontsao/Desktop/Datasets/EarningsCallCleaned/EarningsCall_earningsCall_Biotechnology.csv')

In [14]:
All_content = df_bio['Content'].tolist()
doc_clean_all = [clean(doc).split() for doc in All_content]  

In [15]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      52.464314        1       1 -0.129966 -0.136177
0      40.532631        1       2 -0.174800  0.123446
2       7.003056        1       3  0.304767  0.012731, topic_info=     Category          Freq       Term         Total  loglift  logprob
term                                                                  
55    Default  20533.000000        â¢  20533.000000  30.0000  30.0000
10    Default  17100.000000    current  17100.000000  29.0000  29.0000
372   Default  14308.000000   question  14308.000000  28.0000  28.0000
320   Default   6687.000000       next   6687.000000  27.0000  27.0000
513   Default  14999.000000      think  14999.000000  26.0000  26.0000
5     Default   9332.000000       call   9332.000000  25.0000  25.0000
285   Default   6341.000000       line   6341.000000  24.0000  24.0000
8     Default   9462.000000    company   9462.000000  23.0000  23.0000
3     Default   7243.000000  bloomberg   7243.000000  22.0000  22.0000
41    Default  23721.000000    quarter  23721.000000  21.0000  21.0000
121   Default   4520.000000       come   4520.000000  20.0000  20.0000
14    Default   6431.000000   earnings   6431.000000  19.0000  19.0000
1007  Default   4421.000000      thank   4421.000000  18.0000  18.0000
16    Default   7010.000000   estimate   7010.000000  17.0000  17.0000
54    Default   6449.000000        ytd   6449.000000  16.0000  16.0000
7     Default   8417.000000     change   8417.000000  15.0000  15.0000
42    Default   8837.000000       sale   8837.000000  14.0000  14.0000
448   Default   8418.000000     thanks   8418.000000  13.0000  13.0000
308   Default  10685.000000    million  10685.000000  12.0000  12.0000
33    Default   6842.000000       name   6842.000000  11.0000  11.0000
406   Default  10073.000000    revenue  10073.000000  10.0000  10.0000
50    Default  22823.000000       year  22823.000000   9.0000   9.0000
640   Default   3461.000000        you   3461.000000   8.0000   8.0000
638   Default   3159.000000       okay   3159.000000   7.0000   7.0000
784   Default   7351.000000         â   7351.000000   6.0000   6.0000
225   Default   7219.000000      going   7219.000000   5.0000   5.0000
15    Default   4578.000000        eps   4578.000000   4.0000   4.0000
772   Default   5186.000000        get   5186.000000   3.0000   3.0000
11    Default   3937.000000       date   3937.000000   2.0000   2.0000
623   Default   3426.000000      right   3426.000000   1.0000   1.0000
...       ...           ...        ...           ...      ...      ...
1567   Topic3   1046.390259     please   1488.652222   2.3063  -4.9731
320    Topic3   3466.455811       next   6687.936523   2.0017  -3.7753
29     Topic3    361.355042    matthew    409.731506   2.5332  -6.0364
285    Topic3   3208.891602       line   6341.415039   1.9776  -3.8525
121    Topic3   2439.258057       come   4520.469727   2.0419  -4.1268
1149   Topic3    308.559509       bank    342.626740   2.5541  -6.1943
1007   Topic3   1902.166748      thank   4421.825195   1.8153  -4.3755
638    Topic3   1360.920410       okay   3159.083008   1.8167  -4.7103
331    Topic3    924.214783       open   1804.193604   1.9899  -5.0973
640    Topic3   1354.406860        you   3461.517578   1.7205  -4.7151
372    Topic3   3394.280762   question  14308.396484   1.2201  -3.7964
3      Topic3   2036.159302  bloomberg   7243.973633   1.3897  -4.3074
9828   Topic3    407.222473       eric    592.660156   2.2836  -5.9169
5      Topic3   1934.680054       call   9332.662109   1.0852  -4.3585
8      Topic3   1851.957031    company   9462.937500   1.0277  -4.4022
10     Topic3   2546.939453    current  17100.101562   0.7546  -4.0836
14     Topic3   1474.962280   earnings   6431.016113   1.1863  -4.6298
31     Topic3    525.407104    michael    976.350830   2.0392  -5.6620
33     Topic3   1252.590820       name   6842.081055   0.9609  -4.

In [16]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=4, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      38.227852        1       1 -0.159449 -0.047563
3      31.685539        1       2  0.032003 -0.193147
2      21.229807        1       3 -0.179857  0.154720
1       8.856795        1       4  0.307303  0.085990, topic_info=     Category          Freq        Term         Total  loglift  logprob
term                                                                   
55    Default  22118.000000         â¢  22118.000000  30.0000  30.0000
372   Default  13792.000000    question  13792.000000  29.0000  29.0000
448   Default   7890.000000      thanks   7890.000000  28.0000  28.0000
41    Default  23782.000000     quarter  23782.000000  27.0000  27.0000
513   Default  15312.000000       think  15312.000000  26.0000  26.0000
10    Default  17210.000000     current  17210.000000  25.0000  25.0000
320   Default   6680.000000        next   6680.000000  24.0000  24.0000
285   Default   6297.000000        line   6297.000000  23.0000  23.0000
121   Default   4539.000000        come   4539.000000  22.0000  22.0000
50    Default  22824.000000        year  22824.000000  21.0000  21.0000
1007  Default   4351.000000       thank   4351.000000  20.0000  20.0000
341   Default  18535.000000     patient  18535.000000  19.0000  19.0000
638   Default   3074.000000        okay   3074.000000  18.0000  18.0000
406   Default  10065.000000     revenue  10065.000000  17.0000  17.0000
640   Default   3377.000000         you   3377.000000  16.0000  16.0000
145   Default   4283.000000       could   4283.000000  15.0000  15.0000
512   Default   2712.000000      taking   2712.000000  14.0000  14.0000
558   Default   2969.000000       maybe   2969.000000  13.0000  13.0000
308   Default  10591.000000     million  10591.000000  12.0000  12.0000
784   Default   7445.000000          â   7445.000000  11.0000  11.0000
5     Default   9390.000000        call   9390.000000  10.0000  10.0000
42    Default   8850.000000        sale   8850.000000   9.0000   9.0000
580   Default   4820.000000      really   4820.000000   8.0000   8.0000
230   Default   7940.000000      growth   7940.000000   7.0000   7.0000
287   Default   3423.000000      little   3423.000000   6.0000   6.0000
642   Default   2065.000000         guy   2065.000000   5.0000   5.0000
225   Default   7346.000000       going   7346.000000   4.0000   4.0000
14    Default   6528.000000    earnings   6528.000000   3.0000   3.0000
983   Default   3270.000000         bit   3270.000000   2.0000   2.0000
847   Default   2216.000000          hi   2216.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
2047   Topic4    232.138733       sound    250.304184   2.3486  -6.7137
3630   Topic4    183.438110     stanley    193.739807   2.3693  -6.9492
899    Topic4    198.111328       sachs    210.879272   2.3615  -6.8722
898    Topic4    197.074753     goldman    209.820724   2.3613  -6.8775
558    Topic4   2121.111572       maybe   2969.350098   2.0876  -4.5013
314    Topic4    222.668945      morgan    240.959930   2.3450  -6.7554
121    Topic4   2976.534668        come   4539.541016   2.0019  -4.1625
285    Topic4   3951.118652        line   6297.668945   1.9578  -3.8793
320    Topic4   4104.174805        next   6680.766602   1.9368  -3.8413
640    Topic4   2167.399170         you   3377.348877   1.9804  -4.4798
943    Topic4   1191.454590       guess   1742.348145   2.0439  -5.0781
1567   Topic4   1046.467651      please   1500.987549   2.0633  -5.2079
1007   Topic4   2411.352539       thank   4351.681641   1.8336  -4.3731
502    Topic4   1636.702515       great   2720.895996   1.9157  -4.7606
863    Topic4    724.110962      wanted   1036.458496   2.0654  -5.5761
498    Topic4    660.550537         ask    931.174438   2.0806  -5.6680
145    Topic4   2038.016357       could   4283.491699   1.6812  -4.5413
331    Topic4   1038.548218      

In [17]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      35.254536        1       1 -0.187324 -0.046879
3      31.433210        1       2 -0.132431  0.178143
1      23.317175        1       3 -0.177952 -0.063426
0       5.674918        1       4  0.289770  0.215430
4       4.320169        1       5  0.207937 -0.283268, topic_info=      Category          Freq         Term         Total  loglift  logprob
term                                                                     
55     Default  20809.000000          â¢  20809.000000  30.0000  30.0000
372    Default  13754.000000     question  13754.000000  29.0000  29.0000
10     Default  17059.000000      current  17059.000000  28.0000  28.0000
448    Default   7918.000000       thanks   7918.000000  27.0000  27.0000
41     Default  23793.000000      quarter  23793.000000  26.0000  26.0000
513    Default  15256.000000        think  15256.000000  25.0000  25.0000
8      Default   9452.000000      company   9452.000000  24.0000  24.0000
320    Default   6620.000000         next   6620.000000  23.0000  23.0000
285    Default   6253.000000         line   6253.000000  22.0000  22.0000
50     Default  22801.000000         year  22801.000000  21.0000  21.0000
5      Default   9370.000000         call   9370.000000  20.0000  20.0000
7      Default   8391.000000       change   8391.000000  19.0000  19.0000
16     Default   6992.000000     estimate   6992.000000  18.0000  18.0000
14     Default   6440.000000     earnings   6440.000000  17.0000  17.0000
54     Default   6429.000000          ytd   6429.000000  16.0000  16.0000
3      Default   7306.000000    bloomberg   7306.000000  15.0000  15.0000
121    Default   4485.000000         come   4485.000000  14.0000  14.0000
1007   Default   4361.000000        thank   4361.000000  13.0000  13.0000
42     Default   8868.000000         sale   8868.000000  12.0000  12.0000
341    Default  18589.000000      patient  18589.000000  11.0000  11.0000
640    Default   3372.000000          you   3372.000000  10.0000  10.0000
638    Default   3085.000000         okay   3085.000000   9.0000   9.0000
15     Default   4568.000000          eps   4568.000000   8.0000   8.0000
4455   Default   2321.000000           mm   2321.000000   7.0000   7.0000
512    Default   2706.000000       taking   2706.000000   6.0000   6.0000
784    Default   7472.000000           â   7472.000000   5.0000   5.0000
33     Default   6827.000000         name   6827.000000   4.0000   4.0000
406    Default  10112.000000      revenue  10112.000000   3.0000   3.0000
308    Default  10711.000000      million  10711.000000   2.0000   2.0000
17     Default   4309.000000        event   4309.000000   1.0000   1.0000
...        ...           ...          ...           ...      ...      ...
39      Topic5    261.627808       porges    366.122925   2.8058  -5.8762
1317    Topic5    240.424057       robert    376.099487   2.6944  -5.9607
30      Topic5    253.808090      meacham    419.242981   2.6400  -5.9065
62318   Topic5     94.846115       alfred    125.220367   2.8641  -6.8909
10      Topic5   4008.110840      current  17059.283203   1.6935  -3.1470
14      Topic5   1823.975830     earnings   6440.785156   1.8802  -3.9343
8       Topic5   2384.253906      company   9452.944336   1.7644  -3.6665
29      Topic5    229.345413      matthew    412.691284   2.5544  -6.0079
54      Topic5   1639.901978          ytd   6429.688477   1.7756  -4.0407
4453    Topic5    523.217712          iâ   1338.190186   2.2028  -5.1831
16      Topic5   1682.746826     estimate   6992.945312   1.7174  -4.0149
51      Topic5    160.107025          yee    259.941101   2.6573  -6.3673
3       Topic5   1526.721802    bloomberg   7306.972168   1.5762  -4.1122
7       Topic5   1667.734131       change   8391.376953   1.5261  -4.0239
5       Topic5   1675.620728         call   9370.476562   1.4205  -4.0192
47      Topic5    825.84234

### Semiconductor

Try with all texts within earnings call transcript

In [18]:
# Semiconductor
df_semiconductor = pd.read_csv('/Users/sharontsao/Desktop/Datasets/EarningsCallCleaned/EarningsCall_earningsCall_Semiconductors.csv')

In [19]:
All_content = df_semiconductor['Content'].tolist()
doc_clean_all = [clean(doc).split() for doc in All_content]  

In [20]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      40.120846        1       1  0.178408  0.061708
2      35.937325        1       2 -0.018369 -0.147418
1      23.941826        1       3 -0.160040  0.085711, topic_info=     Category          Freq       Term         Total  loglift  logprob
term                                                                  
38    Default  50372.000000    quarter  50372.000000  30.0000  30.0000
12    Default  24393.000000    current  24393.000000  29.0000  29.0000
463   Default  19869.000000   question  19869.000000  28.0000  28.0000
50    Default  21459.000000        â¢  21459.000000  27.0000  27.0000
578   Default  28244.000000      think  28244.000000  26.0000  26.0000
1253  Default  18202.000000    million  18202.000000  25.0000  25.0000
643   Default  10210.000000     thanks  10210.000000  24.0000  24.0000
1039  Default  12205.000000       next  12205.000000  23.0000  23.0000
1320  Default  16151.000000       were  16151.000000  22.0000  22.0000
42    Default  13845.000000       sale  13845.000000  21.0000  21.0000
20    Default  10075.000000   estimate  10075.000000  20.0000  20.0000
572   Default  10296.000000      thank  10296.000000  19.0000  19.0000
389   Default   7347.000000       okay   7347.000000  18.0000  18.0000
6     Default  13889.000000       call  13889.000000  17.0000  17.0000
631   Default  17134.000000         â  17134.000000  16.0000  16.0000
675   Default  11610.000000     really  11610.000000  15.0000  15.0000
121   Default   9208.000000       cash   9208.000000  14.0000  14.0000
892   Default   8747.000000        you   8747.000000  13.0000  13.0000
49    Default   8838.000000        ytd   8838.000000  12.0000  12.0000
348   Default  13012.000000     margin  13012.000000  11.0000  11.0000
17    Default   8785.000000   earnings   8785.000000  10.0000  10.0000
263   Default  16058.000000      going  16058.000000   9.0000   9.0000
10    Default  18834.000000    company  18834.000000   8.0000   8.0000
1321  Default   8839.000000       weve   8839.000000   7.0000   7.0000
829   Default   5336.000000        guy   5336.000000   6.0000   6.0000
336   Default   7592.000000     little   7592.000000   5.0000   5.0000
501   Default  19571.000000    revenue  19571.000000   4.0000   4.0000
1305  Default  11817.000000      thats  11817.000000   3.0000   3.0000
756   Default   8161.000000       yeah   8161.000000   2.0000   2.0000
4     Default  10124.000000  bloomberg  10124.000000   1.0000   1.0000
...       ...           ...        ...           ...      ...      ...
336    Topic3   5831.990723     little   7592.523438   1.1657  -4.9546
3371   Topic3   2035.423950       opex   2484.870117   1.2300  -6.0073
130    Topic3   4862.173828       come   6620.291992   1.1209  -5.1365
887    Topic3   1322.260498     wanted   1558.017090   1.2655  -6.4387
1504   Topic3   2764.337646        yes   3613.758057   1.1616  -5.7012
633    Topic3   5204.154297      could   7541.305664   1.0586  -5.0685
106    Topic3   6201.097168        bit  10079.643555   0.9438  -4.8933
756    Topic3   5186.046387       yeah   8161.580566   0.9761  -5.0720
259    Topic3   3431.038086       give   5096.110352   1.0339  -5.4851
275    Topic3   3076.312500       half   4455.878418   1.0591  -5.5943
2371   Topic3   4775.265137  inventory   7900.206055   0.9261  -5.1546
348    Topic3   7055.679688     margin  13012.619141   0.8175  -4.7642
621    Topic3   6324.594238      would  12307.931641   0.7637  -4.8736
1358   Topic3   4366.508789       line   7557.843750   0.8809  -5.2440
861    Topic3   4028.457764      great   6829.822754   0.9016  -5.3246
578    Topic3  10772.541992      think  28244.570312   0.4656  -4.3410
317    Topic3   4754.730469       kind   8988.062500   0.7928  -5.1589
38     Topic3  13301.012695    quarter  50372.632812   0.0979  -4.1302
262    Topic3   4640.994629         go   9319.321289   0.7324  -5.

In [21]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=4, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      36.900291        1       1 -0.045906 -0.157261
1      36.593781        1       2 -0.165429 -0.051056
0      19.968040        1       3 -0.161461  0.174682
2       6.537888        1       4  0.372796  0.033635, topic_info=      Category          Freq         Term         Total  loglift  logprob
term                                                                     
50     Default  18082.000000          â¢  18082.000000  30.0000  30.0000
12     Default  23196.000000      current  23196.000000  29.0000  29.0000
38     Default  50377.000000      quarter  50377.000000  28.0000  28.0000
10     Default  18206.000000      company  18206.000000  27.0000  27.0000
463    Default  19825.000000     question  19825.000000  26.0000  26.0000
6      Default  13422.000000         call  13422.000000  25.0000  25.0000
1253   Default  18472.000000      million  18472.000000  24.0000  24.0000
578    Default  28273.000000        think  28273.000000  23.0000  23.0000
20     Default   9572.000000     estimate   9572.000000  22.0000  22.0000
9      Default  11658.000000       change  11658.000000  21.0000  21.0000
4      Default   9672.000000    bloomberg   9672.000000  20.0000  20.0000
49     Default   8356.000000          ytd   8356.000000  19.0000  19.0000
42     Default  13642.000000         sale  13642.000000  18.0000  18.0000
17     Default   8369.000000     earnings   8369.000000  17.0000  17.0000
643    Default  10182.000000       thanks  10182.000000  16.0000  16.0000
48     Default  39194.000000         year  39194.000000  15.0000  15.0000
3229   Default   4473.000000        laser   4473.000000  14.0000  14.0000
1039   Default  12187.000000         next  12187.000000  13.0000  13.0000
1320   Default  16399.000000         were  16399.000000  12.0000  12.0000
572    Default  10298.000000        thank  10298.000000  11.0000  11.0000
892    Default   8736.000000          you   8736.000000  10.0000  10.0000
389    Default   7336.000000         okay   7336.000000   9.0000   9.0000
675    Default  11739.000000       really  11739.000000   8.0000   8.0000
121    Default   9310.000000         cash   9310.000000   7.0000   7.0000
631    Default  17193.000000           â  17193.000000   6.0000   6.0000
501    Default  19779.000000      revenue  19779.000000   5.0000   5.0000
18     Default   5538.000000          eps   5538.000000   4.0000   4.0000
829    Default   5339.000000          guy   5339.000000   3.0000   3.0000
348    Default  13045.000000       margin  13045.000000   2.0000   2.0000
32     Default  10760.000000         name  10760.000000   1.0000   1.0000
...        ...           ...          ...           ...      ...      ...
6725    Topic4   1101.682983       nvidia   1124.943604   2.7067  -5.3232
4685    Topic4   1447.578369          car   1537.054443   2.6676  -5.0501
4851    Topic4   1227.489136       gaming   1309.023926   2.6632  -5.2150
4936    Topic4    449.591095  workstation    457.597748   2.7099  -6.2194
3055    Topic4    536.436646           ai    552.647156   2.6978  -6.0428
34      Topic4   1515.710083  participant   1688.159790   2.6198  -5.0041
85434   Topic4   1342.622925        tegra   1552.397705   2.5824  -5.1254
37510   Topic4   1003.513855        fiber   1335.653564   2.4416  -5.4165
12      Topic4   7344.843750      current  23196.589844   1.5775  -3.4260
2937    Topic4   1204.178711    computing   1999.868164   2.2203  -5.2342
10      Topic4   5106.626465      company  18206.617188   1.4563  -3.7895
49      Topic4   2900.993896          ytd   8356.639648   1.6696  -4.3549
20      Topic4   3075.161865     estimate   9572.422852   1.5920  -4.2966
4       Topic4   2969.844971    bloomberg   9672.012695   1.5468  -4.3315
6       Topic4   3388.121582         call  13422.876953   1.3509  -4.1997
17      Topic4   2570.895508     earnings   8369.473633   1.5472  -4.4757
9      

In [22]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      36.898640        1       1  0.140387  0.113207
2      27.753904        1       2  0.173357 -0.052198
0      27.033150        1       3  0.205216 -0.049783
1       5.245827        1       4 -0.258509  0.231118
3       3.068475        1       5 -0.260451 -0.242344, topic_info=      Category          Freq                  Term         Total  loglift  \
term                                                                        
50     Default  18695.000000                   â¢  18695.000000  30.0000   
12     Default  24030.000000               current  24030.000000  29.0000   
463    Default  19611.000000              question  19611.000000  28.0000   
572    Default   9376.000000                 thank   9376.000000  27.0000   
1039   Default  11881.000000                  next  11881.000000  26.0000   
10     Default  18452.000000               company  18452.000000  25.0000   
38     Default  50662.000000               quarter  50662.000000  24.0000   
892    Default   8108.000000                   you   8108.000000  23.0000   
643    Default   9831.000000                thanks   9831.000000  22.0000   
4      Default   9855.000000             bloomberg   9855.000000  21.0000   
6      Default  13667.000000                  call  13667.000000  20.0000   
578    Default  28575.000000                 think  28575.000000  19.0000   
502    Default   7510.000000                 right   7510.000000  18.0000   
389    Default   7024.000000                  okay   7024.000000  17.0000   
20     Default   9933.000000              estimate   9933.000000  16.0000   
17     Default   8573.000000              earnings   8573.000000  15.0000   
49     Default   8676.000000                   ytd   8676.000000  14.0000   
9      Default  11931.000000                change  11931.000000  13.0000   
1253   Default  18502.000000               million  18502.000000  12.0000   
1358   Default   7398.000000                  line   7398.000000  11.0000   
42     Default  13930.000000                  sale  13930.000000  10.0000   
130    Default   6400.000000                  come   6400.000000   9.0000   
861    Default   6540.000000                 great   6540.000000   8.0000   
32     Default  10731.000000                  name  10731.000000   7.0000   
369    Default   7699.000000                  much   7699.000000   6.0000   
264    Default  10821.000000                  good  10821.000000   5.0000   
1305   Default  11734.000000                 thats  11734.000000   4.0000   
262    Default   9170.000000                    go   9170.000000   3.0000   
48     Default  39804.000000                  year  39804.000000   2.0000   
631    Default  17245.000000                    â  17245.000000   1.0000   
...        ...           ...                   ...           ...      ...   
1994    Topic5    306.948761  consequentialspecial    307.723450   3.4815   
1995    Topic5    303.665710           constitutes    304.440918   3.4814   
2033    Topic5    302.064362          solicitation    302.839172   3.4814   
38462   Topic5   1065.379517                 qorvo   1135.563110   3.4202   
20176   Topic5   1005.896973                filter   1077.277832   3.4154   
2005    Topic5    605.999512             expressed    632.664185   3.4409   
572     Topic5   5814.116211                 thank   9376.330078   3.0061   
1524    Topic5    439.104309               implied    475.192047   3.4050   
892     Topic5   3981.474609                   you   8108.948242   2.7727   
2035    Topic5    372.446228            thirdparty    394.500732   3.4265   
2021    Topic5    746.979431              personal   1069.923340   3.1247   
141     Topic5    394.692780            connection    432.316284   3.3929   
389     Topic5   2198.194580                  okay   7024.347656   2.3222   
502     Topic5   1885.301147         

### Media

In [24]:
# Media
df_media = pd.read_csv('/Users/sharontsao/Desktop/Datasets/EarningsCallCleaned/EarningsCall_earningsCall_Media.csv')

In [25]:
All_content = df_media['Content'].tolist()
doc_clean_all = [clean(doc).split() for doc in All_content]  

In [26]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      51.567196        1       1 -0.122441 -0.128468
0      41.291515        1       2 -0.179882  0.113165
2       7.141282        1       3  0.302323  0.015303, topic_info=      Category          Freq       Term         Total  loglift  logprob
term                                                                   
56     Default  18964.000000        â¢  18964.000000  30.0000  30.0000
482    Default  12319.000000   question  12319.000000  29.0000  29.0000
13     Default  17532.000000    current  17532.000000  28.0000  28.0000
736    Default   8163.000000       next   8163.000000  27.0000  27.0000
662    Default   7171.000000      thank   7171.000000  26.0000  26.0000
585    Default  20588.000000      think  20588.000000  25.0000  25.0000
43     Default  29375.000000    quarter  29375.000000  24.0000  24.0000
666    Default   5656.000000        you   5656.000000  23.0000  23.0000
8      Default   9863.000000       call   9863.000000  22.0000  22.0000
579    Default   7580.000000     thanks   7580.000000  21.0000  21.0000
6      Default   7317.000000  bloomberg   7317.000000  20.0000  20.0000
735    Default   6174.000000         go   6174.000000  19.0000  19.0000
12     Default  12405.000000    company  12405.000000  18.0000  18.0000
21     Default   7017.000000   earnings   7017.000000  17.0000  17.0000
152    Default   4616.000000       come   4616.000000  16.0000  16.0000
690    Default   4079.000000       line   4079.000000  15.0000  15.0000
718    Default  10590.000000         â  10590.000000  14.0000  14.0000
453    Default   3480.000000     please   3480.000000  13.0000  13.0000
37     Default   7170.000000       name   7170.000000  12.0000  12.0000
23     Default   6654.000000   estimate   6654.000000  11.0000  11.0000
55     Default   6635.000000        ytd   6635.000000  10.0000  10.0000
10     Default   8514.000000     change   8514.000000   9.0000   9.0000
507    Default  16154.000000    revenue  16154.000000   8.0000   8.0000
574    Default   4946.000000       take   4946.000000   7.0000   7.0000
429    Default   2838.000000   operator   2838.000000   6.0000   6.0000
737    Default   2934.000000       okay   2934.000000   5.0000   5.0000
638    Default  12620.000000       well  12620.000000   4.0000   4.0000
287    Default   8736.000000      going   8736.000000   3.0000   3.0000
733    Default   2744.000000      ahead   2744.000000   2.0000   2.0000
48     Default   6589.000000       sale   6589.000000   1.0000   1.0000
...        ...           ...        ...           ...      ...      ...
1545    Topic3    278.885345    implied    287.826324   2.6077  -6.3813
482     Topic3   6102.790039   question  12319.226562   1.9369  -3.2956
453     Topic3   2145.068848     please   3480.416260   2.1553  -4.3412
152     Topic3   2515.525146       come   4616.613281   2.0321  -4.1819
429     Topic3   1675.845825   operator   2838.526855   2.1123  -4.5881
56      Topic3   7106.712891        â¢  18964.087891   1.6578  -3.1433
737     Topic3   1675.861206       okay   2934.165527   2.0792  -4.5881
28      Topic3    736.137207    jessica   1037.200928   2.2964  -5.4107
16576   Topic3    571.178528      craig    745.534485   2.3729  -5.6644
733     Topic3   1413.975586      ahead   2744.654053   1.9760  -4.7580
735     Topic3   2422.906738         go   6174.810547   1.7038  -4.2194
579     Topic3   2524.430908     thanks   7580.063477   1.5398  -4.1784
33      Topic3    901.832275    michael   1650.682617   2.0348  -5.2077
1268    Topic3    443.413849    opinion    536.574768   2.4486  -5.9176
6       Topic3   1856.608398  bloomberg   7317.672363   1.2677  -4.4856
8       Topic3   2052.497803       call   9863.586914   1.0695  -4.3853
2622    Topic3    698.612305       john   1238.889038   2.0664  -5.4630
13      Topic3   2528.091797    current  17532.453125   0.7027  -4.1769
21      Topic3 

In [27]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=4, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      49.714325        1       1  0.065652  0.164023
2      27.887505        1       2  0.169969 -0.035332
3      15.900588        1       3  0.110948 -0.107894
0       6.497588        1       4 -0.346570 -0.020797, topic_info=     Category          Freq       Term         Total  loglift  logprob
term                                                                  
56    Default  19029.000000        â¢  19029.000000  30.0000  30.0000
482   Default  12228.000000   question  12228.000000  29.0000  29.0000
13    Default  17572.000000    current  17572.000000  28.0000  28.0000
585   Default  20495.000000      think  20495.000000  27.0000  27.0000
736   Default   8200.000000       next   8200.000000  26.0000  26.0000
662   Default   7387.000000      thank   7387.000000  25.0000  25.0000
6     Default   7338.000000  bloomberg   7338.000000  24.0000  24.0000
8     Default   9909.000000       call   9909.000000  23.0000  23.0000
639   Default  13320.000000       were  13320.000000  22.0000  22.0000
43    Default  29675.000000    quarter  29675.000000  21.0000  21.0000
579   Default   7460.000000     thanks   7460.000000  20.0000  20.0000
21    Default   7036.000000   earnings   7036.000000  19.0000  19.0000
666   Default   5738.000000        you   5738.000000  18.0000  18.0000
12    Default  12453.000000    company  12453.000000  17.0000  17.0000
735   Default   6196.000000         go   6196.000000  16.0000  16.0000
152   Default   4636.000000       come   4636.000000  15.0000  15.0000
690   Default   4112.000000       line   4112.000000  14.0000  14.0000
453   Default   3496.000000     please   3496.000000  13.0000  13.0000
718   Default  10539.000000         â  10539.000000  12.0000  12.0000
23    Default   6670.000000   estimate   6670.000000  11.0000  11.0000
55    Default   6651.000000        ytd   6651.000000  10.0000  10.0000
429   Default   2865.000000   operator   2865.000000   9.0000   9.0000
638   Default  12681.000000       well  12681.000000   8.0000   8.0000
37    Default   7180.000000       name   7180.000000   7.0000   7.0000
287   Default   8698.000000      going   8698.000000   6.0000   6.0000
10    Default   8540.000000     change   8540.000000   5.0000   5.0000
574   Default   4959.000000       take   4959.000000   4.0000   4.0000
733   Default   2762.000000      ahead   2762.000000   3.0000   3.0000
737   Default   2922.000000       okay   2922.000000   2.0000   2.0000
507   Default  16230.000000    revenue  16230.000000   1.0000   1.0000
...       ...           ...        ...           ...      ...      ...
482    Topic4   5384.828613   question  12228.662109   1.9135  -3.3263
152    Topic4   2452.666016       come   4636.479492   2.0970  -4.1127
56     Topic4   7163.228516        â¢  19029.265625   1.7567  -3.0409
26     Topic4    745.060852      jason   1043.133301   2.3972  -5.3042
429    Topic4   1604.282837   operator   2865.096191   2.1538  -4.5372
666    Topic4   2653.844482        you   5738.812988   1.9625  -4.0339
28     Topic4    733.841492    jessica   1042.234375   2.3829  -5.3194
1538   Topic4    452.079956  expressed    555.869080   2.5271  -5.8038
1268   Topic4    442.270630    opinion    540.500671   2.5332  -5.8257
733    Topic4   1389.409058      ahead   2762.290039   2.0466  -4.6810
737    Topic4   1435.878784       okay   2922.368164   2.0231  -4.6481
735    Topic4   2366.150879         go   6196.024414   1.7711  -4.1486
579    Topic4   2131.237305     thanks   7460.420410   1.4808  -4.2532
33     Topic4    837.186646    michael   1660.417114   2.0490  -5.1876
6      Topic4   1864.547852  bloomberg   7338.533691   1.3636  -4.3869
8      Topic4   1963.839600       call   9909.831055   1.1151  -4.3350
13     Topic4   2500.376709    current  17572.275391   0.7839  -4.0935
21     Topic4   1587.075684   earnings   7036.156738   1.2446  -4.5480
2622   Topic

In [28]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.

dictionary = corpora.Dictionary(doc_clean_all)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean_all]
corpus = [dictionary.doc2bow(doc) for doc in doc_clean_all]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50, minimum_probability=0)

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      42.017593        1       1 -0.103083 -0.119708
3      30.856546        1       2 -0.155252  0.082088
0      13.468594        1       3 -0.123720 -0.166488
2       7.821617        1       4  0.024512  0.279765
1       5.835647        1       5  0.357543 -0.075658, topic_info=      Category          Freq       Term         Total  loglift  logprob
term                                                                   
56     Default  18533.000000        â¢  18533.000000  30.0000  30.0000
13     Default  18245.000000    current  18245.000000  29.0000  29.0000
482    Default  12043.000000   question  12043.000000  28.0000  28.0000
585    Default  21280.000000      think  21280.000000  27.0000  27.0000
43     Default  29278.000000    quarter  29278.000000  26.0000  26.0000
736    Default   8096.000000       next   8096.000000  25.0000  25.0000
662    Default   7587.000000      thank   7587.000000  24.0000  24.0000
860    Default   7244.000000   customer   7244.000000  23.0000  23.0000
579    Default   7430.000000     thanks   7430.000000  22.0000  22.0000
12     Default  12723.000000    company  12723.000000  21.0000  21.0000
21     Default   7433.000000   earnings   7433.000000  20.0000  20.0000
718    Default  11011.000000         â  11011.000000  19.0000  19.0000
666    Default   5798.000000        you   5798.000000  18.0000  18.0000
735    Default   6116.000000         go   6116.000000  17.0000  17.0000
152    Default   4640.000000       come   4640.000000  16.0000  16.0000
8      Default   9708.000000       call   9708.000000  15.0000  15.0000
507    Default  16654.000000    revenue  16654.000000  14.0000  14.0000
55     Default   6897.000000        ytd   6897.000000  13.0000  13.0000
23     Default   6884.000000   estimate   6884.000000  12.0000  12.0000
690    Default   3867.000000       line   3867.000000  11.0000  11.0000
6      Default   7665.000000  bloomberg   7665.000000  10.0000  10.0000
54     Default  29864.000000       year  29864.000000   9.0000   9.0000
176    Default   4544.000000      could   4544.000000   8.0000   8.0000
10     Default   8563.000000     change   8563.000000   7.0000   7.0000
453    Default   3427.000000     please   3427.000000   6.0000   6.0000
639    Default  13375.000000       were  13375.000000   5.0000   5.0000
391    Default  11141.000000    million  11141.000000   4.0000   4.0000
1226   Default   4635.000000    service   4635.000000   3.0000   3.0000
1613   Default   4719.000000       cash   4719.000000   2.0000   2.0000
404    Default   4972.000000       much   4972.000000   1.0000   1.0000
...        ...           ...        ...           ...      ...      ...
453     Topic5   2020.128174     please   3427.228516   2.3126  -4.1993
3594    Topic5    333.995880      vijay    377.083496   2.7198  -5.9991
152     Topic5   2383.117188       come   4640.625977   2.1747  -4.0340
482     Topic5   4923.082520   question  12043.675781   1.9466  -3.3085
26      Topic5    646.314331      jason    902.142334   2.5077  -5.3389
429     Topic5   1437.186646   operator   2759.083008   2.1890  -4.5397
28      Topic5    677.689453    jessica   1018.127686   2.4342  -5.2915
733     Topic5   1395.759277      ahead   2827.772949   2.1351  -4.5690
735     Topic5   2284.053711         go   6116.385742   1.8562  -4.0765
33      Topic5    830.411194    michael   1443.239014   2.2885  -5.0883
1799    Topic5   1295.941772       weâ   3100.270996   1.9689  -4.6432
13      Topic5   3472.013916    current  18245.650391   1.1820  -3.6577
16576   Topic5    486.707581      craig    754.969360   2.4022  -5.6225
2622    Topic5    654.748657       john   1247.197021   2.1968  -5.3259
21      Topic5   1845.865967   earnings   7433.875977   1.4481  -4.2895
8       Topic5   1821.502686       call   9708.071289   1.1679  -4.3028
12      Topic5   2052.726074    company  12723.1718